In [ ]:
!pip install fuzzywuzzy
!pip install transformers

import transformers
from datetime import datetime
import pandas as pd
from fuzzywuzzy import process  # For fuzzy matching

# Initialize the text-generation pipeline with multiple models
def init_chatbot():
    print("Initializing chatbot...")

    # DialogGPT model for conversation
    dialog_model_name = "microsoft/DialoGPT-medium"
    dialog_tokenizer = transformers.AutoTokenizer.from_pretrained(dialog_model_name, use_fast=True)
    dialog_model = transformers.AutoModelForCausalLM.from_pretrained(dialog_model_name)
    dialog_chatbot = transformers.pipeline(
        "text-generation",
        model=dialog_model,
        tokenizer=dialog_tokenizer,
        pad_token_id=dialog_tokenizer.eos_token_id
    )

    # Additional GPT-2 model for handling different types of responses
    additional_model_name = "gpt2"
    additional_tokenizer = transformers.AutoTokenizer.from_pretrained(additional_model_name, use_fast=True)
    additional_model = transformers.AutoModelForCausalLM.from_pretrained(additional_model_name)
    additional_chatbot = transformers.pipeline(
        "text-generation",
        model=additional_model,
        tokenizer=additional_tokenizer,
        pad_token_id=additional_tokenizer.eos_token_id
    )

    print("Chatbot is ready to talk!")
    return dialog_chatbot, additional_chatbot

# Handle greetings based on time of day
def handle_greetings():
    current_hour = datetime.now().hour
    if 5 <= current_hour < 12:
        return "Good morning!"
    elif 12 <= current_hour < 17:
        return "Good afternoon!"
    elif 17 <= current_hour < 21:
        return "Good evening!"
    else:
        return "Hello there!"

# Existing data extracted from the document
data = {
    'Question': [
        'Who is Onfon Mobile?',
        'How do I get a phone with Onfon Mobile?',
        'Where are your devices located?',
        'What do I need when getting an Onfon device?',
        'Who qualifies for an Onfon Mobile device?',
        'What are the Payment Plans available on the Onfon Mobile?',
        'Can I change my payment plan?',
        'How do I pay for my phone loan?',
        'What are the requirements for making loan payments?',
        'What happens if I do not make payments on time.',
        'Is my phone locked immediately or is there a grace period to pay?',
        'Can I get more than one phone on loan?',
        'Can I make payments for another number?',
        'Is there a till number for making payments?',
        'Can I purchase Airtime with Onfon Mobile?',
        'What should I do if I lose my phone and haven’t finished paying for the loan?',
        'Does my phone have a warranty?',
        'Can my phone be repaired after it gets spoilt?',
        'How can I contact Onfon Mobile?',
        'Are there other services under Onfon Mobile?',
        'What phones do you sell?'
    ],
    'Answer': [
        'Onfon Mobile is a device financing company that helps to issue smartphones to customers on loan services.',
        'To get a smartphone via Onfon Mobile, register with us via *797#. After dialing *797#, go through the registration process, and you shall receive a message telling you whether you qualify for a smartphone(s) or not. Customer registers for the service. Onfon Mobile scores the customer, and if they qualify, they are shown a list of devices they qualify for. If you receive a message that you qualify for a phone, you can dial *797# again to see devices you qualify for. Make your loan payment. Purchase airtime.',
        'You can access the Onfon Mobile service(s) from Safaricom shops across the country as well as some Safaricom Dealership shops within the country.',
        'You will need your national ID for verification within the shop where you shall be going to get your preferred device.',
        'Everyone who meets the credit scoring requirements can access an Onfon Mobile phone.',
        'For a device bought under Onfon Mobile, there are payments that are to be made depending on the payment plan that the customer wants. We currently have the 12-month loan, and customers can change the plan after they are enrolled in the 12-month payment plan.',
        'Yes, you can change your payment plan on our Onfon Mobile app.',
        'To make payments for your phone loan, you may use our USSD by dialing *797# and choosing option 2. Alternatively, a customer may use the Onfon Mobile Application to make their payment. Within the application, choose the pay option, and the M-Pesa SDK shall pop up, and the customer can make their payment. You may also pay using M-Pesa by choosing the Lipa na Mpesa option, selecting the Paybill option, entering Business Number 622645, and entering the borrower\'s ID/Phone Number.',
        'The customer should ensure that they make their payments as per their payment plans, failure to do this will lead to their phones being locked.',
        'Your device will be locked if you do not pay your installments on time.',
        'You do have a grace period of about 10–15 minutes before your phone is locked.',
        'No, you can only be issued one phone to pay for on loan. Once you finish your payment for that phone, you can get access to another phone.',
        'Yes, you can make payments for another number.',
        'We have a business paybill number. Simply use the Paybill option on M-Pesa and key in the business number 622645, then use the customer ID number or phone number as the account number, then enter the amount to be paid, finish off with your Mpesa password, and you will have made your payment.',
        'Yes, you can. To purchase airtime, simply dial *797# and choose the buy airtime option on the USSD.',
        'If you lose your phone, report the case to the police immediately. Acquire an OB number and a police abstract. Then share the scanned documents with customercare@onfonmobile.com.',
        'Yes, the phone has a warranty of 1 year from the manufacturer. Terms and conditions apply.',
        'Yes, for phone damages under warranty, simply visit the nearest service center and get the assistance you need with repair.',
        'For further inquiries, reach out to Onfon Mobile via our email customercare@onfonmobile.com or call our customer care line 0709491700 or find us on all our social media pages as Onfon Mobile.',
        'Yes, we have a referral code program in place that allows our customers to earn money by referencing other customers to us. You earn a commission for the customer who registers and purchases from us.',
        'Our phone options cut across all Android devices.'
    ]
}

# Convert existing data to DataFrame
existing_df = pd.DataFrame(data)

# Fuzzy matching function to match queries with existing questions
def fuzzy_match_query(query, df, threshold=80):
    result = process.extractOne(query, df['Question'])
    print(f"Fuzzy match result: {result}")  # Debugging print statement
    if result:
        best_match, score, _ = result  # Unpacking three values
        if score >= threshold:
            matched_answer = df[df['Question'] == best_match]['Answer'].values[0]
            return matched_answer
    return None

# Generate a response using the chatbot models
def generate_response(chatbots, query):
    dialog_chatbot, additional_chatbot = chatbots

    # Try DialogGPT first
    response_dialog = dialog_chatbot(query, max_length=50, num_return_sequences=1)
    response_dialog_text = response_dialog[0]['generated_text']

    # If DialogGPT response is too generic, try the additional model
    if "sorry" in response_dialog_text.lower() or len(response_dialog_text) < 20:
        response_additional = additional_chatbot(query, max_length=50, num_return_sequences=1)
        response_additional_text = response_additional[0]['generated_text']
        return response_additional_text

    return response_dialog_text

# Combine everything into the chat function
def chat(chatbots, query):
    greeting = handle_greetings()

    # Check if the query matches any existing question using fuzzy matching
    matched_answer = fuzzy_match_query(query, existing_df)
    if matched_answer:
        return f"{greeting} {matched_answer}"

    # Generate a response using the chatbot models
    response = generate_response(chatbots, query)
    return f"{greeting} {response}"

# Initialize the chatbot
chatbots = init_chatbot()

# Example usage
query = "How do I get a phone with Onfon Mobile?"
response = chat(chatbots, query)
print(response)


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Initializing chatbot...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Chatbot is ready to talk!
Fuzzy match result: ('How do I get a phone with Onfon Mobile?', 100, 1)
Good morning! To get a smartphone via Onfon Mobile, register with us via *797#. After dialing *797#, go through the registration process, and you shall receive a message telling you whether you qualify for a smartphone(s) or not. Customer registers for the service. Onfon Mobile scores the customer, and if they qualify, they are shown a list of devices they qualify for. If you receive a message that you qualify for a phone, you can dial *797# again to see devices you qualify for. Make your loan payment. Purchase airtime.


In [ ]:
# Initialize the chatbot
chatbots = init_chatbot()

# Interactive loop for asking questions
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break
    response = chat(chatbots, query)
    print(f"Chatbot: {response}")

Initializing chatbot...
Chatbot is ready to talk!


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Fuzzy match result: ('Who is Onfon Mobile?', 30, 0)
Chatbot: Good morning! hi and welcome to the party
Fuzzy match result: ('What are the requirements for making loan payments?', 90, 8)
Chatbot: Good morning! The customer should ensure that they make their payments as per their payment plans, failure to do this will lead to their phones being locked.
